In [45]:

import pandas as pd
import numpy as np
from collections import Counter
import os.path as osp
import json
import matplotlib.pyplot as plt
import pickle

# Print all variables on their own lines
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

pd.set_option('display.max_rows', 1000)


In [46]:
"""First run dataset saving script that saves all per-sample entries in the stream."""
train_usersummary_file = "/home/matthiasdelange/sftp_remote_projects/ContextualOracle_Matthias/results/ego4d_action_recog/summarize_streams/logs/2022-08-04_18-05-40_UID8f0427cb-8048-47d5-b70e-bc3878a1cb3a/dataset_entries_train_ego4d_LTA_train_usersplit_10users_pickle.ckpt"

FILE_TO_ANALYZE = train_usersummary_file


with open(FILE_TO_ANALYZE, 'rb') as f:
    ds = pickle.load(f)
#     analyze_obj = json.load(p)

'First run dataset saving script that saves all per-sample entries in the stream.'

In [48]:
def ds_to_user_dfs(ds):
    ret = {}
    for user, user_entries in ds.items():
        # Do all for actions/verbs/nouns
        user_df = pd.json_normalize(user_entries)  # Convert to DF
    
        # Create action column
        def label_fn(x):
            assert len(x) == 2, "Need two columns to merge"
            if not isinstance(x[0],list):
                assert not isinstance(x[1],list)
                return f"{x[0]}-{x[1]}"

            return [f"{l}-{r}" for l,r in zip(x[0],x[1])]

        user_df['action_label'] = user_df.loc[:,('verb_label','noun_label')].apply(label_fn,axis=1)
        ret[user] = user_df
        print("Created action_label column")
    return ret

In [71]:
def plot_user_histogram_grid(dfs):
    """"""
    cols = ['verb_label','noun_label','action_label']
    nb_users = len(dfs)
    fig, ax = plt.subplots(nb_users, len(cols), figsize=(15, 30), dpi=600)

    
    for row_idx, (user_id, user_df) in enumerate(dfs.items()):
        for col_idx, col in enumerate(cols):
            cnt = Counter(user_df[col].tolist())

            col_sorted = sorted([(k,v) for k,v in cnt.items() ] ,key=lambda x: x[1], reverse=True)
            vals_sorted = [x[0] for x in col_sorted]
            cnts_sorted = [x[1] for x in col_sorted]

            print(f"Freqs for {col}: {cnts_sorted}")
            print(f"Labels for {col}: {vals_sorted}")

            y_axis= cnts_sorted
            x_axis= list(range(len(cnts_sorted)))
            nb_samples = sum(cnts_sorted)
            nb_mins = int(nb_samples*2.1/60)
            plot_subplot(ax[row_idx,col_idx], x_axis, y_axis, title=f'USER {user_id} - {col}| #={nb_samples}|mins={nb_mins}')
#     plt.suptitle(f'Stream sample histogram plot: Verb/noun/action freq {user_id}')
    
    fig.tight_layout() 
    plt.show()
    plt.clf()

    
# Barchart API: https://matplotlib.org/stable/api/_as_gen/matplotlib.pyplot.bar.html#matplotlib.pyplot.bar
def plot_subplot(sub_ax, x_axis, y_vals, title,ylabel=None,xlabel=None, 
                  grid=False,yerror=None,xerror=None, y_labels=None, x_labels=None,bar_align='edge',barh=False,
                 figsize=(12, 6), log=False, interactive=False,x_minor_ticks=None):
    max_val = max(y_vals)
    my_cmap = plt.get_cmap("plasma")
#     fig = plt.figure(figsize=figsize, dpi=600) # So all bars are visible!
#     ax=plt.subplot()
    
    sub_ax.bar(x_axis, height=y_vals,color=my_cmap.colors, align=bar_align,yerr=yerror,width=0.9,log=log)

    if x_minor_ticks is not None:
        sub_ax.set_xticks(x_minor_ticks, minor=True)


    if x_labels:
        plt.xticks(x_axis, x_labels, rotation='vertical')
    if y_labels:
        plt.yticks(y_vals, y_labels)
    
    sub_ax.set_ylim(None,max_val*1.01)
    sub_ax.set_xlim(None,None)
#     sub_ax.set_xlabel(xlabel)
#     sub_ax.set_ylabel(ylabel)
#     plt.title(title)
    sub_ax.set_title(title)
    sub_ax.grid(grid, which='both')

In [39]:
from collections import Counter



    
def plot_user_summary(user_id: str, user_df):
    """"""

    # Plot accum 2s-clips per action + total length over all actions
    # plot_total_action_length_histogram()
    user_histogram(user_df, user_id)

    # Plot Number of actions instead of length + mention max per-clip length (e.g. 2s)

    # Plot like EPIC KITCHENS a normalized bar with same action same color
    # Also do for verbs/nouns separately
    return user_df




def user_histogram(user_df, user_id):
    for col in ['verb_label','noun_label','action_label']:
        cnt = Counter(user_df[col].tolist())
        
        col_sorted = sorted([(k,v) for k,v in cnt.items() ] ,key=lambda x: x[1], reverse=True)
        vals_sorted = [x[0] for x in col_sorted]
        cnts_sorted = [x[1] for x in col_sorted]
        
        print(f"Freqs for {col}: {cnts_sorted}")
        print(f"Labels for {col}: {vals_sorted}")
        
        y_axis= cnts_sorted
        x_axis= list(range(len(cnts_sorted)))
        plot_barchart(x_axis, y_axis, title=f'{col} frequency plot user {user_id}',ylabel='Input clip count',xlabel=f"{col} - Sorted")

    # Collect plots and show together
    plt.show()
    plt.clf()
    
# Barchart API: https://matplotlib.org/stable/api/_as_gen/matplotlib.pyplot.bar.html#matplotlib.pyplot.bar
def plot_barchart(x_axis, y_vals, title,ylabel,xlabel='User-ID', 
                  grid=False,yerror=None,xerror=None, y_labels=None, x_labels=None,bar_align='edge',barh=False,
                 figsize=(12, 6), log=False, interactive=False,x_minor_ticks=None):
    max_val = max(y_vals)
    my_cmap = plt.get_cmap("plasma")
    fig = plt.figure(figsize=figsize, dpi=600) # So all bars are visible!
    ax=plt.subplot()
    
    if not barh:
        bars = plt.bar(x_axis, height=y_vals,color=my_cmap.colors, align=bar_align,yerr=yerror,width=0.9,log=log)
    else:
        bars = plt.barh(y_vals, width=x_axis,color=my_cmap.colors, align=bar_align,xerr=xerror,height=0.9,log=log)
    
    if x_minor_ticks is not None:
        ax.set_xticks(x_minor_ticks, minor=True)


    if x_labels:
        plt.xticks(x_axis, x_labels, rotation='vertical')
    if y_labels:
        plt.yticks(y_vals, y_labels)
        

    
    plt.ylim(None,max_val*1.01)
    plt.xlim(None,None)
    plt.xlabel(xlabel)
    plt.ylabel(ylabel)
    plt.title(title)
    plt.grid(grid, which='both')
        

    

In [72]:
# MAIN SCRIPT
dfs = ds_to_user_dfs(ds) # Preprocess dataframes

# Grids of histograms
plot_user_histogram_grid(dfs)

# Make grid
# for user, user_df in dfs.items():
#     user_df = 
#     user_df.head(n=100)
#     break


Created action_label column
Created action_label column
Created action_label column
Created action_label column
Created action_label column
Created action_label column
Created action_label column
Created action_label column
Created action_label column
Created action_label column
Freqs for verb_label: [93, 89, 79, 48, 44, 42, 36, 27, 26, 25, 23, 21, 21, 14, 13, 11, 10, 7, 7, 4, 4, 4, 3, 3, 3, 3, 2, 2, 1, 1, 1, 1]
Labels for verb_label: [92, 65, 19, 28, 0, 49, 86, 64, 9, 34, 98, 50, 5, 100, 35, 47, 67, 99, 60, 2, 11, 37, 94, 6, 69, 14, 1, 90, 52, 17, 61, 78]
Freqs for noun_label: [356, 35, 24, 22, 22, 20, 20, 16, 15, 13, 12, 11, 11, 10, 9, 8, 7, 7, 6, 6, 6, 5, 3, 3, 3, 3, 3, 2, 2, 2, 2, 1, 1, 1, 1]
Labels for noun_label: [97, 120, 228, 41, 364, 34, 233, 129, 119, 411, 160, 49, 158, 26, 403, 105, 35, 439, 18, 159, 223, 145, 52, 284, 328, 148, 17, 219, 27, 391, 468, 211, 254, 59, 331]
Freqs for action_label: [74, 58, 45, 37, 35, 27, 25, 22, 17, 15, 12, 11, 11, 11, 10, 10, 9, 8, 8, 7, 7, 7,

Freqs for action_label: [59, 51, 51, 50, 40, 35, 26, 22, 20, 19, 18, 17, 16, 15, 15, 14, 14, 13, 12, 12, 11, 11, 11, 10, 10, 10, 10, 10, 9, 9, 9, 9, 9, 9, 9, 8, 8, 8, 7, 7, 7, 7, 7, 7, 7, 7, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
Labels for action_label: ['92-352', '92-472', '65-352', '65-472', '65-353', '92-353', 

Freqs for noun_label: [40, 24, 18, 18, 13, 7, 6, 6, 4, 4, 3, 3, 3, 3, 3, 3, 2, 1, 1, 1, 1]
Labels for noun_label: [300, 104, 224, 255, 158, 313, 41, 232, 245, 46, 38, 457, 165, 361, 205, 52, 287, 297, 150, 59, 105]
Freqs for action_label: [21, 18, 17, 15, 6, 6, 6, 5, 4, 4, 4, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 2, 2, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1]
Labels for action_label: ['51-104', '65-300', '92-300', '51-224', '62-232', '99-255', '9-158', '51-300', '92-46', '65-313', '112-158', '61-224', '92-313', '49-38', '65-457', '92-165', '92-41', '0-104', '33-205', '62-255', '49-52', '19-255', '67-255', '49-158', '65-255', '50-41', '92-245', '92-361', '92-287', '65-297', '11-150', '50-59', '0-245', '65-41', '50-361', '16-245', '11-105']


<Figure size 432x288 with 0 Axes>